<a href="https://colab.research.google.com/github/Hariprasad-KK/Adv_DataAnalytics_MiniProjects/blob/main/Copy_of_final_bert_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
!pip install googletrans==4.0.0-rc1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
from googletrans import Translator
import pandas as pd
import time
import traceback
import tweepy
import re



In [24]:
api_key = "YK6eUfcBsF8C0pM5PknXkS4Or"
api_secret = "Kjp7OqHCqSzYarPMapCI1cJXsz6y9oSJwknydGBEy0ELxvIkBT"
access_token = "1336204649989165057-cYfKk0s2uUoaIyDXwzZ8IHzhe3CghB"
access_secret = "osVdZpG0WJOxQjYgHqj11ArZgeymrwT4QnwcILYTU2y2V"

auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
tweets, text_query = [], 'spanish'
count = 20

try:
    for tweet in api.search(q=text_query, count=count, result_type='recent',
                           include_entities=True,
                           monitor_rate_limit=True, 
                           wait_on_rate_limit=True):
        # print(f"Raw tweet: {tweet}")
        tweet_text_full, tweet_text_translated = "", ""

        status = api.get_status(id = tweet.id, tweet_mode="extended")
        try:
            tweet_text_full = status.retweeted_status.full_text
        except AttributeError:  # Not a Retweet
            tweet_text_full = status.full_text
            
        try:
            tr = Translator()
            
            if tweet.lang and tweet.lang not in ['en']:


                print(f"Raw tweet: {tweet_text_full}")
                    
                if tweet.text:
                    translated = tr.translate(tweet_text_full)
                    if translated:
                        tweet_text = translated.__dict__()["text"]
            else:
                tweet_text_translated = tweet_text_full
                
        except Exception as e:
            traceback.print_exc()
            pass
        
        tweets.append(( tweet_text_full))
        
        df_tr = pd.DataFrame(tweets, columns = ['Tweets'])
        time.sleep(3)
        
    print("Completed.")
    
except BaseException as e:
    traceback.print_exc() 

In [ ]:
df_tr.head(20)

In [ ]:
tweet

In [ ]:
path = "/content/drive/MyDrive/Caps/"

df_tr.to_csv(path + 'tweets_full_translated.csv')

In [ ]:
import numpy as np
from googletrans import Translator
from googletrans import LANGUAGES
import pandas as pd
# Reading the Live tweets data
LiveTweets=pd.read_csv( '/content/drive/MyDrive/Caps/tweets_full_translated.csv', encoding='latin')
print(LiveTweets.shape)




In [ ]:
translator = Translator()
print(LiveTweets)

In [ ]:
Translated_Tweets=[]
for element in LiveTweets['Tweets']:
  Translated_Tweets.append(translator.translate(element).text)
LiveTweets['Translated_Tweets']=Translated_Tweets
print(LiveTweets)

In [ ]:
LiveTweets.head(10)
LiveTweets.drop("Unnamed: 0",axis=1)
LiveTweets.Tweets=LiveTweets.Tweets.str.replace(' ',' ')
LiveTweets.Tweets =LiveTweets.Tweets.str.lstrip()
LiveTweets.Tweets = LiveTweets.Tweets.str.rstrip()
LiveTweets.Tweets = LiveTweets.Tweets.str.strip()
#LiveTweets['Tweets'] = LiveTweets['Tweets'].str.replace('\W', ' ', regex=True)
print(LiveTweets)



In [ ]:
#removing mentions
LiveTweets['Tweets'] = LiveTweets['Tweets'].str.replace(r'\s*@\w+', '', regex=True)
LiveTweets['Tweets'] = LiveTweets['Tweets'].str.replace(r'\s*#\w+', '', regex=True)
LiveTweets['Tweets'] = LiveTweets['Tweets'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')
LiveTweets['Tweets'] = LiveTweets['Tweets'].str.replace('\d+', '')
LiveTweets['Tweets'] = LiveTweets['Tweets'].str.replace('\W', ' ', regex=True)
print(LiveTweets)

In [ ]:
# installing the library 'transformers' which contains BERT implementation
!pip install transformers

# installing the library tensorflow
!pip install tensorflow

In [ ]:
# importing the pipeline module
from transformers import pipeline

# Downloading the sentiment analysis model
SentimentClassifier = pipeline("sentiment-analysis")

In [ ]:
# Calling the sentiment analysis function for 3 sentences
SentimentClassifier(["I hope we get all these concepts! Its killing the neurons of our brain",
                     "We had a nice experience in this trip",
                     "Houston we have a problem"
])

In [ ]:
# Defining a function to call for the whole dataframe
def FunctionBERTSentiment(inpText):
 return(SentimentClassifier(inpText)[0]['label'])

# Calling the function
FunctionBERTSentiment(inpText="Houston we have a problem")

In [ ]:
# Calling BERT based sentiment score function for every tweet
LiveTweets['Sentiment']=LiveTweets['Tweets'].apply(FunctionBERTSentiment)
LiveTweets.head(10)


In [ ]:
# Visualizing the overall sentiment distribution
import matplotlib.pyplot as plt
fig, subPlot =plt.subplots(nrows=1, ncols=2, figsize=(10,4))
fig.suptitle("Sentiment analysis of Live Tweets")

# Grouping the data
GroupedData=LiveTweets.groupby('Sentiment').size()

# Creating the charts
GroupedData.plot(kind='bar', ax=subPlot[0], color=['crimson', 'lightblue'])
GroupedData.plot(kind='pie', ax=subPlot[1], colors=['crimson', 'lightblue'])

In [ ]:
import os
os.remove('/content/drive/MyDrive/Caps/tweets_full_translated.csv')